In [1]:
from dask.distributed import Client, LocalCluster    
import os
import datetime as dt
import pandas as pd
import requests
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.dataframe_functions import roll_time_series
import xgboost as xgb
import numpy as np
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import matplotlib
import dask.dataframe as dd
from dask.dataframe import from_pandas
from dask.distributed import progress
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import multiprocessing
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

In [2]:
cpu_cores = multiprocessing.cpu_count()
os.makedirs('dask-scraFONh', exist_ok=True)
cluster = LocalCluster(local_directory='dask-scraFONh', n_workers= cpu_cores)
client = Client(cluster)

In [ ]:
def get_proj_ids(coins, project_url_df):
    list_id = []
    for coin in coins:
        list_id.append(project_url_df.loc[project_url_df['symbol']== coin,'project_id'].item())
    return list_id

def get_project_url_df():
    url = "https://api.tokenterminal.com/v2/projects"
    headers = {
    "accept": "application/json",
    "authorization": TOKENTERMINAL_API_KEY
    }
    response = requests.get(url, headers=headers)
    list_of_dicts= eval(response.text.replace('null', 'None'))
    return pd.DataFrame(list_of_dicts['data'])

def get_specific_urls(coins, project_url_X):
    spec_urls = project_url_df.loc[project_url_df['symbol'].isin(coins), 'url'].tolist()
    return spec_urls

def get_df_of_OCA(coins, date, project_url_df):
    list_dd = []
    headers = {
        "accept": "application/json",
        "authorization": TOKENTERMINAL_API_KEY
        }
    for url in get_specific_urls(coins, project_url_df):
        response = requests.get(f'https://api.tokenterminal.com{url}/metrics', headers=headers)
        data = response.text.replace("null", "None")
        raw_data_string = data[response.text.find("['")+9:response.text.find("']")]
        try: raw_data = eval(raw_data_string)
        except: continue
        list_dd.append(pd.DataFrame(raw_data))
    FON = list_dd[0]
    count = 0
    for list in list_dd:
        if count == 0: 
            count = count +1
            continue
        else:
            FON= pd.merge(left = FON, right= list_dd[count], how = 'outer', suffixes = (None, coins[count]))
            count = count + 1
            continue
    # FON.drop(['project_name'], axis=1, inplace = True)
    FON['timestamp']=FON['timestamp'].apply(pd.to_datetime)
    FON= FON[(FON['timestamp'] >= date)]
    # npartitions = FON['project_name'].nunique()
    return FON.set_index('project_name')

def roll(FON, timeshift, coins):
    FON.dropna(subset = 'forecast', inplace=True)
    FON.dropna(subset = 'price', inplace=True)
    FON[FON.columns.drop(['timestamp', 'project_id','forecast'])] = FON.drop(['timestamp', 'project_id','forecast'], axis =1).ffill()
    X = FON 
    FON_rolled = roll_time_series(X, column_id='project_id', column_sort='timestamp', max_timeshift=timeshift,  n_jobs =1)
    return FON_rolled.drop('project_id', axis=1)

def add_inf_int(X, Sup_features):
    X = X.merge(Sup_features, how = 'left', on = 'timestamp', suffixes = (None, None))
    X[X.columns.drop]
    # X.replace('<NA>', np.NaN, regex=True)
    return X
def merge_BTC_feat(part, BTC_feat):
    if part['project_id'].unique() != 'bitcoin':
        X = part.merge(BTC_feat, how = 'left', on = 'timestamp', suffixes=(None, '_BTC'))
    else: 
        X = part
    return X
def calc_log_returns_and_vol(FON, days_forecast, target):
    FON.sort_values(by= 'timestamp', inplace=True)
    FON['LOG_R'] = np.log(FON['price'])- np.log(FON['price'].shift(days_forecast))
    FON['EWM_VOL_R'] = FON['LOG_R'].ewm(alpha=0.01).std()
    FON['price_volatility'] = FON['price'].rolling(window=days_forecast).std() 
    FON['returns_volatility'] = FON['LOG_R'].rolling(window=days_forecast).std() 
    if target == 'price_volatility':
        FON['forecast'] = FON['price_volatility'].shift(-days_forecast)
    elif target == 'LOG_R':
        FON['forecast'] =  np.log(FON['price'].shift(-days_forecast))- np.log(FON['price'])
    elif target == 'price':
        FON['forecast'] = FON['price'].shift(-days_forecast)
    elif target == 'returns_volatility':
         FON['forecast'] = FON['returns_volatility'].shift(-days_forecast)
    else:
        print('Invalid target')
    return FON

In [4]:
# new_coins = ['COW','SAFE','GNO']
# days_forecast= 15
# # coins = ['SOL','BNB','ETH', 'ADA', 'BTC']
# date = pd.to_datetime(dt.datetime.today() -pd.Timedelta(days=days_forecast), utc =True) - pd.Timedelta(150, 'W')
# timeshift = 365
# project_url_df = get_project_url_df()  
# project_ids = get_proj_ids(coins, project_url_df)
# names = sorted(project_ids)

SyntaxError: invalid syntax (589223481.py, line 1)

In [11]:
coins = ['COW','SAFE','GNO', 'ETH']
date = pd.to_datetime("jan 1 2021").tz_localize('UTC')
project_url_df = get_project_url_df()  
project_ids = get_proj_ids(coins, project_url_df)
names = sorted(project_ids)
FON = get_df_of_OCA(coins, date, project_url_df)

In [20]:
FON.index.unique()

Index(['CoW Protocol', 'Gnosis', 'Safe'], dtype='object', name='project_name')

In [21]:
COW= FON.loc['CoW Protocol'].dropna(how='all').dropna(axis=1, how='all')
SAFE =FON.loc['Safe'].dropna(how='all').dropna(axis=1, how='all')
GNO = FON.loc['Gnosis'].dropna(how='all').dropna(axis=1, how='all')

In [ ]:
for project in FON.index.unique():
    FON.loc[project]['price_MA(50)']= FON.loc[project]['price'].rolling(window=50).mean()

In [18]:
# target = 'price_volatility'
# FON = get_df_of_OCA(coins, date, project_url_df)
# FON= FON.map_partitions(calc_log_returns_and_vol, days_forecast, target, enforce_metadata=False).map_partitions(add_inf_int, Sup_features, align_dataframes=False).persist()
# BTC_feat = FON.partitions[names.index('bitcoin')][['LOG_R', 'EWM_VOL_R', 'price_volatility', 'returns_volatility', 'timestamp']].set_index('timestamp')
# FON = FON.map_partitions(merge_BTC_feat, BTC_feat, align_dataframes=False).persist()

In [17]:
# Sup_features = pd.read_csv(r"C:\Crypto_Price_Action\5YF_inflation_expectations.csv").merge(pd.read_csv(r"C:\Crypto_Price_Action\5Y_FFER.csv"), on = 'DATE')
# Sup_features['DATE'] = pd.to_datetime(Sup_features['DATE'], utc=True)
# Sup_features.rename({'DATE':'timestamp','T5YIFR':'5Y_inflation_expectations_US', 'DFF':'Federal_Funds_Rate'}, inplace=True, axis=1)
# Sup_features[Sup_features.drop('timestamp', axis=1).columns] = Sup_features[Sup_features.drop('timestamp', axis=1).columns].apply(pd.to_numeric, errors='coerce')

In [19]:
FON.partitions[1].compute()['5Y_inflation_expectations_US']

0       2.13
1       2.06
2       2.12
3        NaN
4        NaN
        ... 
1059    2.36
1060     NaN
1061     NaN
1062    2.35
1063     NaN
Name: 5Y_inflation_expectations_US, Length: 1064, dtype: float64

In [20]:
relevance= abs(FON.partitions[1].drop(['timestamp', 'project_id'], axis =1).compute().ffill().dropna(how='all').dropna(subset='forecast').corr()['forecast'])
relevance.sort_values(ascending=False, inplace=True)
relevance

forecast                        1.000000
token_supply_circulating        0.623881
expenses                        0.607527
token_incentives                0.607527
price                           0.566214
market_cap_circulating          0.559916
market_cap_fully_diluted        0.559916
Federal_Funds_Rate              0.544949
fees                            0.497156
transaction_fee_average         0.495534
afpu                            0.483089
revenue                         0.478072
fees_supply_side                0.474101
arpu                            0.465561
block_time                      0.432167
price_volatility                0.429549
transaction_count               0.428943
transactions_per_second         0.428943
tokenholders                    0.410806
contract_deployers              0.408052
price_volatility_BTC            0.376834
earnings                        0.363967
pf_circulating                  0.355012
pf_fully_diluted                0.355012
token_trading_vo

In [22]:
list_rev_var = []
for (key,value) in relevance.items(): 
    if value >= 0.10: list_rev_var.append(key)
# list_rev_var.remove('LOG_R_BTC')
# list_rev_var.remove('price_std')
# list_rev_var.remove('market_cap_circulating')
# list_rev_var.remove('market_cap_fully_diluted')
# list_rev_var.remove('market_cap_circulating')
# list_rev_var.remove('pf_fully_diluted')
# list_rev_var.remove('token_turnover_fully_diluted')
# list_rev_var.remove('token_turnover_circulating')
# list_rev_var.remove('ps_circulating')
# list_rev_var.remove('ps_fully_diluted')
# list_rev_var.remove('pf_circulating')
# list_rev_var.remove('transaction_fee_average')
# list_rev_var.remove('token_incentives')
# list_rev_var.remove('fees')
list_rev_var

['forecast',
 'token_supply_circulating',
 'expenses',
 'token_incentives',
 'price',
 'market_cap_circulating',
 'market_cap_fully_diluted',
 'Federal_Funds_Rate',
 'fees',
 'transaction_fee_average',
 'afpu',
 'revenue',
 'fees_supply_side',
 'arpu',
 'block_time',
 'price_volatility',
 'transaction_count',
 'transactions_per_second',
 'tokenholders',
 'contract_deployers',
 'price_volatility_BTC',
 'earnings',
 'pf_circulating',
 'pf_fully_diluted',
 'token_trading_volume',
 'contracts_deployed',
 'ps_circulating',
 'ps_fully_diluted',
 'user_mau',
 'user_wau',
 '5Y_inflation_expectations_US',
 'user_dau']

In [27]:
variables = list_rev_var
variables = pd.Series(variables).unique().tolist()
variables.append('timestamp')
variables.append('id')
rolled = FON.map_partitions(roll, timeshift, coins, enforce_metadata=False).persist()
ETH = rolled.partitions[1].compute()[variables]

Rolling: 100%|██████████| 1/1 [00:00<00:00, 17.80it/s]


In [28]:
df = ETH.dropna()
y= df.groupby('timestamp')[['forecast', 'timestamp', 'id']].first().set_index('id')  
X = df.drop(['forecast'], axis=1)
X= X.merge(y['forecast'], how='left', on=['id'])
X = X.dropna(subset=['forecast'], axis =0)
X[X.drop(['id','timestamp','forecast'], axis =1).columns] =X[X.drop(['id','timestamp','forecast'], axis =1).columns].fillna(method='ffill', axis=0)
X = X.dropna()
# default_fc_parameters=EfficientFCParameters()
FC = extract_relevant_features(X.drop('forecast', axis =1), X.groupby('id')['forecast'].last(), column_id='id', column_sort='timestamp', n_jobs= cpu_cores, hypotheses_independent=False, ml_task='regression', fdr_level=0.001, default_fc_parameters=EfficientFCParameters())
ETH = FC.merge(y[['forecast','timestamp']], how='left', left_index=True, right_index=True).set_index('timestamp')

c:\Crypto_Price_Action\trading_venv\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  if await self.run_code(code, result, async_=asy):
Feature Extraction: 100%|██████████| 100/100 [24:32<00:00, 14.73s/it] 


In [29]:
ETH_rel_table= calculate_relevance_table(ETH.drop('forecast', axis =1), ETH['forecast'], n_jobs= cpu_cores).sort_values(by='p_value', ascending= True)
ETH_rel_table.head(500)

,feature,type,p_value,relevant
feature,,,,
arpu__ar_coefficient__coeff_8__k_10,arpu__ar_coefficient__coeff_8__k_10,real,1.411185e-144,True
block_time__permutation_entropy__dimension_7__tau_1,block_time__permutation_entropy__dimension_7__...,real,2.549837e-143,True
"user_mau__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8","user_mau__change_quantiles__f_agg_""mean""__isab...",real,6.220105e-142,True
afpu__minimum,afpu__minimum,real,1.705554e-140,True
"user_mau__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6","user_mau__change_quantiles__f_agg_""mean""__isab...",real,4.441416e-140,True
...,...,...,...,...
"Federal_Funds_Rate__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2","Federal_Funds_Rate__change_quantiles__f_agg_""m...",real,4.202029e-100,True
user_dau__ratio_beyond_r_sigma__r_0.5,user_dau__ratio_beyond_r_sigma__r_0.5,real,4.415905e-100,True
"fees__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","fees__change_quantiles__f_agg_""mean""__isabs_Tr...",real,5.758304e-100,True


In [32]:
ETH = ETH.merge((FON.partitions[1].compute().set_index('timestamp')[target]).rename('drift'), on = 'timestamp', how='left', suffixes=(None, None))
ETH[['forecast', 'drift']].tail(50)

,forecast,drift
timestamp,,
2024-04-01 00:00:00+00:00,189.603869,122.177330
2024-04-02 00:00:00+00:00,208.859042,132.551901
2024-04-03 00:00:00+00:00,217.729591,113.090386
2024-04-04 00:00:00+00:00,225.179906,118.795548
2024-04-05 00:00:00+00:00,226.708480,123.780949
2024-04-06 00:00:00+00:00,226.590287,120.996553
2024-04-07 00:00:00+00:00,220.016678,116.894873
2024-04-08 00:00:00+00:00,179.226627,131.067522
2024-04-09 00:00:00+00:00,158.952610,127.048459


In [33]:
xgb_model = xgb.XGBRegressor(device = 'cuda',  verbosity=3, n_jobs= -1,  eval_metric='mae',  n_estimators=7500, early_stopping_rounds=25)
param_dist = {
    'max_depth': [3, 5, 10, 25, 50, 100, 200],
    # 'n_estimators': range(5, 205, 25),
    # 'subsample': stats.uniform(0.5, 0.5),
    'gamma': np.arange(0, 2, 0.001).tolist(),
    # 'num_parallel_tree': [1,3,5,10],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 0.7, 1],
    'colsample_by_tree': np.arange(0.1, 1.0, 0.1).tolist(),
    'feature_selector': ['cyclic', 'greedy', 'thrifty'],
    # 'top_k': range(5, 205, 55),
}
splits= 3
random_search = RandomizedSearchCV(
                                    xgb_model, param_distributions=param_dist, n_iter= 15, refit = 'neg_mean_absolute_error', cv=splits,
                                    verbose=3,
                                    scoring=['r2','neg_mean_absolute_error'], 
                                    n_jobs= 1
                                   )
# scoring params:
# 'explained_variance'
# 'max_error'
# 'neg_mean_absolute_error'
# 'neg_mean_squared_error'
# 'neg_root_mean_squared_error'
# 'neg_mean_squared_log_error'
# 'neg_root_mean_squared_log_error'
# 'neg_median_absolute_error'
# 'r2'
# 'neg_mean_poisson_deviance
# 'neg_mean_gamma_deviance'
# 'neg_mean_absolute_percentage_error'
# 'd2_absolute_error_score'
# 'd2_pinball_score'
# 'd2_tweedie_score'

In [35]:
name='ethereum'
X= ETH
# X = SF[name].dropna(subset=['forecast']).set_index('timestamp')
X_train, X_test, Y_train, Y_test = train_test_split(X.drop('forecast', axis=1), X['forecast'], test_size= 0.2, shuffle=False)

In [36]:
opti_xgbmodel = random_search.fit(X_train, Y_train, eval_set = [(X_test, Y_test)], verbose=3)
print(f"{name} Best set of hyperparameters: ", opti_xgbmodel.best_params_)
print(f"{name} Best score: ", opti_xgbmodel.best_score_)
Y_pred= pd.Series(opti_xgbmodel.predict(X_test), X_test.index).rename('prediction')
pd.concat([Y_pred, Y_test.rename("realized"), X_test['VOL'].rename("current")], axis=1).plot(title= f"{name}", xlabel= 'date', ylabel = f'forecast_{days_forecast}_days')


Fitting 3 folds for each of 15 candidates, totalling 45 fits
[21:29:42] ======== Monitor (0): HostSketchContainer ========
[21:29:42] AllReduce: 0.038945s, 1 calls @ 38945us

[21:29:42] MakeCuts: 0.069084s, 1 calls @ 69084us

[21:29:44] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[21:29:44] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\tree\updater_gpu_hist.cu:744: [GPU Hist]: Configure
[21:29:44] ======== Monitor (0):  ========
[21:29:44] InitCompressedData: 0.001187s, 1 calls @ 1187us



c:\Crypto_Price_Action\trading_venv\Lib\site-packages\xgboost\callback.py:354: UserWarning: [21:29:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "colsample_by_tree", "feature_selector" } are not used.

  self.starting_round = model.num_boosted_rounds()


[21:29:45] ======== Monitor (0):  ========
[21:29:45] InitCompressedData: 0.001157s, 1 calls @ 1157us

[0]	validation_0-mae:74.89271
[3]	validation_0-mae:74.88857
[6]	validation_0-mae:74.86867
[9]	validation_0-mae:74.86021
[12]	validation_0-mae:74.87050
[15]	validation_0-mae:74.88871
[18]	validation_0-mae:74.93633
[21]	validation_0-mae:74.96662
[24]	validation_0-mae:75.01687
[27]	validation_0-mae:75.06507
[30]	validation_0-mae:75.13000
[33]	validation_0-mae:75.18234
[35]	validation_0-mae:75.21549
[21:30:18] ======== Monitor (0): Learner ========
[21:30:18] Configure: 0.086842s, 1 calls @ 86842us

[21:30:18] EvalOneIter: 0.072502s, 36 calls @ 72502us

[21:30:18] GetGradient: 0.010443s, 36 calls @ 10443us

[21:30:18] PredictRaw: 0.000155s, 36 calls @ 155us

[21:30:18] UpdateOneIter: 32.0172s, 36 calls @ 32017241us

[21:30:18] ======== Monitor (0): GBTree ========
[21:30:18] BoostNewTrees: 31.9915s, 36 calls @ 31991501us

[21:30:18] CommitModel: 3.1e-05s, 36 calls @ 31us

[21:30:18] =====

c:\Crypto_Price_Action\trading_venv\Lib\site-packages\xgboost\sklearn.py:1168: UserWarning: [21:30:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  predts = self.get_booster().inplace_predict(


[21:30:21] ======== Monitor (0): HostSketchContainer ========
[21:30:21] AllReduce: 0.126998s, 1 calls @ 126998us

[21:30:21] MakeCuts: 0.21642s, 1 calls @ 216420us

[21:30:22] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[21:30:22] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\tree\updater_gpu_hist.cu:744: [GPU Hist]: Configure
[21:30:22] ======== Monitor (0):  ========
[21:30:22] InitCompressedData: 0.001531s, 1 calls @ 1531us



...

c:\Crypto_Price_Action\trading_venv\Lib\site-packages\xgboost\callback.py:354: UserWarning: [21:50:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "colsample_by_tree", "feature_selector" } are not used.

  self.starting_round = model.num_boosted_rounds()


[21:50:11] ======== Monitor (0):  ========
[21:50:11] InitCompressedData: 0.001538s, 1 calls @ 1538us

[0]	validation_0-mae:68.56245
[3]	validation_0-mae:69.14572
[6]	validation_0-mae:69.75481
[9]	validation_0-mae:70.18380
[12]	validation_0-mae:70.56939
[15]	validation_0-mae:71.04653
[18]	validation_0-mae:71.40858
[21]	validation_0-mae:71.84228
[24]	validation_0-mae:72.08195
[25]	validation_0-mae:72.14083
[21:51:41] ======== Monitor (0): Learner ========
[21:51:41] Configure: 0.013061s, 1 calls @ 13061us

[21:51:41] EvalOneIter: 0.049507s, 26 calls @ 49507us

[21:51:41] GetGradient: 0.009706s, 26 calls @ 9706us

[21:51:41] PredictRaw: 0.000169s, 26 calls @ 169us

[21:51:41] UpdateOneIter: 90.0147s, 26 calls @ 90014685us

[21:51:41] ======== Monitor (0): GBTree ========
[21:51:41] BoostNewTrees: 90.003s, 26 calls @ 90002955us

[21:51:41] CommitModel: 3.4e-05s, 26 calls @ 34us

[21:51:41] ======== Device 0 Memory Allocations:  ========
[21:51:41] Peak memory usage: 11464MiB
[21:51:41] Nu

c:\Crypto_Price_Action\trading_venv\Lib\site-packages\xgboost\callback.py:354: UserWarning: [21:51:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "colsample_by_tree", "feature_selector" } are not used.

  self.starting_round = model.num_boosted_rounds()


[21:51:51] ======== Monitor (0):  ========
[21:51:51] InitCompressedData: 0.001276s, 1 calls @ 1276us

[0]	validation_0-mae:63.95830
[3]	validation_0-mae:66.91821
[6]	validation_0-mae:71.08343
[9]	validation_0-mae:74.08493
[12]	validation_0-mae:75.79536
[15]	validation_0-mae:78.07542
[18]	validation_0-mae:78.49459
[21]	validation_0-mae:78.78305
[24]	validation_0-mae:78.65776
[25]	validation_0-mae:78.87627
[21:51:54] ======== Monitor (0): Learner ========
[21:51:54] Configure: 0.002026s, 1 calls @ 2026us

[21:51:54] EvalOneIter: 0.050837s, 26 calls @ 50837us

[21:51:54] GetGradient: 0.010933s, 26 calls @ 10933us

[21:51:54] PredictRaw: 0.000173s, 26 calls @ 173us

[21:51:54] UpdateOneIter: 1.33845s, 26 calls @ 1338453us

[21:51:54] ======== Monitor (0): GBTree ========
[21:51:54] BoostNewTrees: 1.32541s, 26 calls @ 1325413us

[21:51:54] CommitModel: 3.5e-05s, 26 calls @ 35us

[21:51:54] ======== Device 0 Memory Allocations:  ========
[21:51:54] Peak memory usage: 11464MiB
[21:51:54] Num

KeyError: 'VOL'